In [1]:
import cv2
import numpy as np

# Load the pre-trained Haar Cascade classifier for license plate detection
plate_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_russian_plate_number.xml')

# Function to detect license plates within a given frame
def detect_license_plates(frame, car_boxes):
    license_plates = []

    for idx, (x, y, w, h) in car_boxes.iterrows():
        # Extract the region of interest (ROI) corresponding to the car
        roi = frame[y:y+h, x:x+w]

        # Convert the ROI to grayscale (Haar Cascades work better on grayscale images)
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        # Use the classifier to detect license plates within the ROI
        plates = plate_cascade.detectMultiScale(gray_roi, scaleFactor=1.1, minNeighbors=4, minSize=(30, 30))

        # Iterate over the detected plates and adjust coordinates relative to the full frame
        for (px, py, pw, ph) in plates:
            # The detected plate coordinates are relative to the ROI, so we need to adjust them
            absolute_plate_position = (x + px, y + py, pw, ph)
            license_plates.append(absolute_plate_position)

    return license_plates

# Example usage within your video processing loop
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Assume car_boxes is a DataFrame with car bounding boxes for the current frame
        # car_boxes = detect_cars(frame)  # This should be your car detection code

        # Detect license plates in the current frame
        plates = detect_license_plates(frame, car_boxes)

        # Draw rectangles around the detected license plates
        for (x, y, w, h) in plates:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Display the frame with the detected plates
        cv2.imshow('License Plate Detection', frame)

        # Break the loop if the user presses the 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

process_video('/content/sample_data/GX1300015.mp4')